In the US, all financial transaction consumers make are tracked and aggregated into features that describe their financial history. These features can then be used in order to predict how likely a person is to pay back their debts. Many US-based platforms offer unsecured loans, meaning that if a consumer stops paying while there is still outstanding debt (this is called a **charge off** or **defaulting** on the loan) the platform has no legal means for getting their money back, but the consumers' credit score is damaged. Charge-offs are the main risk lenders face, as they greatly decrease the loan's (or more generally, the loan portfolio's) total yield.

In this exercise we will use these credit features in order to predict which loans are likely to CO.

# Loading the data

First we will start by loading the data we will work with during this exercise.  
The data contains information about each loan.  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Load the dataset and take a look at it

In [ ]:
from pagayapro.paths.data_paths import ASSIGNMENT4_DATA
import os

In [ ]:
data = pd.read_parquet(os.path.join(ASSIGNMENT4_DATA,"prosper_data.parquet"))

In [ ]:
data.head()

In [ ]:
data.shape

# Exploratory Data Analysis

So as you did in the previous exercise, the purpose of the next section is getting to know the data and analyzing it.  
You need to understand the data and what is missing in order to find which data preparation steps are required.

In this exercise, we are training a classification model to predict whether or not a loan will repay in full. For this we have 2 columns, co_mob, and co_amount.
**co_mob, stands for: "Charge Off Month On Book".**  
A loan is considered a "charge-off" (AKA "default") if the person stopped paying but still has outstanding debt.  
The "month on book" (mob) is the number of months since the originataion of the loan.  
CO_mob (the charge-off month), is the first month after which there are no more payments (e.g., if someone pays in the first month, misses a payment in the second month, then pays months 3-5 and stops paying completely, his co_mob would be 6, not 2).
co_amount is the amount of outstanding debt lost due to the charge-off.

The main purpose of this exercise is to build a charge-off classification model, predicting which loans will not fully repay their debt.
So, the first thing to do is get to know our 'co_mob' feature.  
Let's plot its distribution in a histogram.

Note, the histogram of CO_MOB is not the probabilty of CO in that mob. If a loan was not charged-off (it was paid in full or pre-paid), its co_mob would be NaN and it would not appear in this graph. Therefore, this graph describes $ P(\text{co_mob} | \text{the loan is a charge-off}) $.

As pointed out above, a loan that did not CO will have a nan co_mob. Let's use that to make our target and drop it from the data

## Features types

Present the data types of each feature

Let's take a look at the non-numeric features (you can use pd.DataFrame.select_dtypes); what are they?

So, we have 3 non-numeric features: borrower_city, issue_date and occupation. Let's handle them.

### Borrower city

How many loans come from each city? 

This is a categorical feature with almost all distinct values (~10,000 values in 40,000 rows), so it is unlikely that it can be useful in predictions. Also, it's not nice to discriminate based on ones geography, let's drop it!

### Issue date

Issue date is ordinal, so we can label encode it (you can use sklearn.preprocessing.LabelEncoder or OrdinalEncoder)

### Occupation

Occupation is not ordinal, but we can target encode it. That is, for each category, replace it with the numeric value of the mean of the target over all elements that belong to that category. However, we first need to split into train and test in order to compute the encoder only on the train set.

## Split to train and test

As you already know, we need to split our data to train and test.
So.. let's make a 70/30 split!

Set you training and test sets to be X_train, y_train, X_test and y_test.

Find the number of rows and columns in your training and test sets:

### Back to the occupation column

Now that we have a train set we can target encode the occupation feature. Compute the encoding on the train set.

Use the encoder to transform occupation in both the train and test sets.

# Logistic Regression - Assumptions

As with the linear regression model, the logistic regression model also has assumptions that need to be met:
1. The target variable is binary.
2. There is no multicollinearity among the features.
3. A linear relationship between the features and the logit of the target.  
 $target \sim Bernouli(p(X))$ where $logit(p_{\beta}(X)) = X\beta$ or, equivalently, $p_{\beta}(X) = logit^{-1}(X\beta)=\frac{\exp(X\beta)}{1+\exp(X\beta)}.$

## Exploring the target variable

Let's check how many charge offs we have:

Remember: True refers to charge-off, and False refers to no charge-off.

What is the percentage of loans with CO?

Compute the (absolute) correlations between each feature and the target

Okay, so co_amount is, unsurprisingly, very correlated with the label. For the moment, though, let's leave it inside the trainig set and see how it effects our results.

Plot the distributions of the next two most correlated features, conditioned on the target varible (i.e. one plot for each class in that target; you can use `hue=target` when using the seaborn suite). 

## Multicollinearity

You are already familiar with the concept of multicollinearty from assignment 3.  

Reminder:
* Multicollinearity occurs when two or more explanatory variables are highly correlated to each other,\
  such that they do not provide unique or independent information in the regression model.  
  If the degree of correlation is high enough between variables, it can cause problems when fitting and interpreting the model. 

Multicollinearity can significantly impair the performance of logistic regression, so let's test for it and remove it

(You can use seaborn.clustermap to vizualize the correlation matrix)

In the heatmap we can see several features that are highly correlated to others. There are more involved ways for choosing which features to keep, but in this case we will simply drop those features which are more correlated to other features as well. Select a set of features that are strongly correlated to other features and drop them.

Make sure the correlation map looks closer to diagonal now.

# Cleaning the data

Until now we got to know our data and checked if they meet the required assumptions.  
Now, let's start cleaning the data.

Find out the percentage of missing values (i.e. NaNs) in your dataset, and print them in ascending order. 

How many missing values are the in the dataset? What's the percentage of rows without any missing features?

If everything went smoothly, there should be no missing values here, so.. nothing left to do :)

## Outliers

If there are outliers, you can choose one of the following ways:
1. Use methods (such as XGBoost) which handle outliers inherently.
2. Remove them.
3. Replace them with a value like the mean/median.
4. Simply keep them in order to learn from them (but make a note about this when reporting the regression results).

Let's focus on two features we understand and find outliers in them:
1) loan_amnt - the amount of the loan in USD.
2) listing_term - the number months the loan was taken for.

In order to find outliers we will plot a box plot for each feature.

There seem to be many outliers with large loan amounts. However, we do expect loan amount to have some very large values, and it is unlikely to expect it to be symmetric (why?).

We can, however, assume that the loan amount behaves log-normal (which means its Log behaves normally), therefore it might be more insightful to plot the Log of the feature. Plot a boxplot of the logarithm of SaleAmt.

This looks much better! 

Let's explore how many outliers we have in this feature. 
In general outlier is not a well-defined notion. One possible defition in our case would be loans whose loan amount is noticably higher or smaller than expected. For example in the case of a normal distribution, values more then $3*std$ from the mean are often considered outliers. A useful rule of thumb for finding outliers in many cases is by computing the interquantile range (IQR, i.e. the difference between the third and first quartile) and finding all loans whose log(loan amount) is more then some constant times the IQR from a given percentile.

Compute the IQR of the log of loan amount.

Compute the number of loans in X_train whose log(loan_amount) is more than $1.5*IQR$ below the 25th percentile of the feature.

Compute the number of loans in X_train whose log(loan_amount) is more than $1.5*IQR$ above the 75th percentile of the feature.

Cool, it seems that there are no outliers according to this specific definition.

Replace the loan_amnt feature by its log:

# Preprocessing

We did most of the data preparation while working.  
The last thing we need to do is to handle categorical features (usually by creating dummy variables or label encoding).  
However, we already know there are no categorical features in our training set so our data is ready! 

# Model train

So, now that our data is ready, we can create and train our model. As we discussed in Section 3, we will use a logistic regression model for this purpose.
Use the `sklearn.linear_model.LogisticRegression` package to create your model. As we've seen in Assignment 1, regression models' performance improve if the data is standardized before fitting. In order to add a standartization step to your model you can use `sklearn.preprocessing.StandardScaler` and use `sklearn.pipeline.PipeLine` in order to run the two processes in sequence.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

Fit your model to the training data

Let's check our model's performance. Our measure of accuracy would be the percentage of items in the test set for which the prediction is identical to the true value (i.e. the percantage of loans for which model predicted True for a CO loan or False for a non-CO loan).

You can either compute this value directly, or use `sklearn.metrics.accuracy_score`.

If all went according to plan, this should be way too good to be true, and make us suspicious. 
Let's have a closer at the trained model. Use `sklearn.linear_model.LogisticRegression`'s coef_ attribute in order to see the regression coefficient corresponding to each feature. By default - the coefficients appear in the same order as the corresponding columns appear in X_train.

Find a nice way to represent the coefficients and their respective column names and print it. How do the coefficients look?

Oh no! co_amount is not supposed to be a feature, it represents the performance of the loan and leaks information about the target (which we obviously won't have in real life before the loan is financed)! This is not surprising- we already saw that it is highly correlated with the target variable.

Let's fix it. Drop the co_amount column from the training and test sets.

Good, now let's retrain and see what we get. Fit the model to the new training set (without co_amount) and assess its performance, as above.

80% accuracy sure seems like a good start. HOWEVER, we saw that there are ~20% charge offs, so a model that predicts all False would essentially perform the same.
Let's look at a confusion matrix of our predictions vs. the true labels, to see if this is the case. Create a confusion matrix using `sklearn.metrics.plot_confusion_matrix`.

Yes, so it seems that our model predicts pretty much all as non-CO. In fact, let's compare to **really** 
predicting non-CO for all loans:

In [ ]:
metrics.accuracy_score(y_test, [False] * len(y_test))

Looks like even the accuracy of always predicting False is almost as good as our model. This is not great..

Moreover, **and this is very important**, in our case not all errors are equally important. A loan with 5% interest rate will yield ~16% of the original loan amount after 3 years ($(1+0.05)^3 -1 = 15.8$), if it repays in full. However, a loan that COs could cost us up to 100%! Therefore, it is **much more important** for us to correctly identify CO's than non-CO's.

Let's see how the accuracy looks if we weighted the COs as 5x (and it is plausible to give it even higher weights). Use `skelearn.metrics.accuracy_score`'s sample_weight parameter to add a weight of 5 to each True prediction of the model.

## Using a weighted model

Of course, the lower accuracy score in the last section is not the model's fault, because we didn't train it to predict with sample weights. But we can retrain the model and give COs higher weights then non-CO loans - let's see what that looks like

Create a linear regerssion model as above with a weight of 5 to each CO loan and 1 for non-CO loans. You can use `sklearn.linear_model.LogisticRegression`'s class_weights parameter for this.

What is the model's accuracy score?

Let's see the precentage of false positive and true negatives using a confusion matrix.

Looks like our model now predicts many more loans as CO than before. Now that we see that changing the weights improve desirable properties of the model, how do we know how to find the *optimal* weights? In fact, in our case we might be able to bypass this problem.

Our logistic regression model predicts the probability of each loan charging off and, by default, flags the loan as CO if it has more than 50% probability of charging off (and non-CO otherwise). Instead of trying various weights and seeing which loans have a probability > 0.5 under these weights, we can decide to label a loan as CO even for a predicted 25%. Or 10%. Or any other threshold. We can then plot the Reciever-Operator Curve (ROC) which tells us the TPR (True Positive Rate) vs. FPR (False Positive Rate) induced by each choice of threshold.

First, in order to see that, indeed, changing the weights is equivalent to changing the threshold instead, let's plot the predicted probabilities for the unwighted model compared with the weighted model and look at the correlation of predictions for both models. 
To obtain the predicted probabilities (instead of the T/F prediction) use the models' predict_proba attribute.

Create a scatter plot of the predicted probabilities of the unweighted model vs those of the weighted model.

Compute the Spearman rank correlation between the predicted probabilities of the two models. A Spearman correlation is a good choice for this type of comparison, because it evaluates the degree to which the models agree about ranking the loans with respect to their probability of CO.

In [ ]:
from scipy.stats import spearmanr


Cool, so we indeed get very good correlation. 

Now let's look at the ROC of this model. Use `sklearn.metrics.plot_roc_curve` to create a ROC of the first (unweighted) model.

Now we know what our options are for trading-off FPR and TPR. The question remains - how do we know which tradeoffs are reasonable?
In our case there are other metrics we can use in order to test our model's performance. Our main goal for training this model is to maximize returns, so it makes sense to evaluate the performance of the model by measuring the performance of our portfolio. We will use [IRR](https://www.investopedia.com/terms/i/irr.asp) to measure the portfolio's yield.

# Computing IRR of different portfolios

In order to compute the IRR of our portfolio we need to get the cashflow (i.e. the payments made for each loan) for our portfolion. We have provided you with a dataframe of precalculated cashflows indexed according to your data.

In [ ]:
cashflows = pd.read_parquet(os.path.join(ASSIGNMENT4_DATA,"prosper_cashflows.parquet"))

The following function receives a dataframe of cashflows and a list of indices and returns the IRR of the portfolio given by these indices.

In [ ]:
import numpy_financial as npf
def get_portfolio_irr(cashflows, indices):
    """
    cashflows- a dataframe of cashflows 
    indices- pd.Series of T/F or a list of indices
    
    output: the yearly irr of the dataframe cashflows.loc[indices]
    """
    return (((npf.irr(cashflows.loc[indices].sum()) + 1) ** 12) - 1) * 100

Compute the IRR of the test set:

Now compute the IRR of all the loans in the test set which were predicted as non-CO:

This is a small improvement, let's see what the x5 weights model looks like:

This is a HUGE improvement! However, we haven't considered how much we had to reduce our volume to reach this return. Let's see what the effect is- compute the percentage of loans for the x5 weights model which managed are flagged as non-CO

So this cost us giving up more than 50% of the portfolio. 

If we have a given amount of money we want to invest we might be willing to settle for a slightly lower return in order to increase the amount of money yielding this return. We can make a plot similar to the ROC, but trading off the IRR on the y-axis and the deployed volume on the x-axis. This will allow us to evaluate these metrics, and interpret them from a buisness standpoint.

Let's have a closer look at this trade-off. Use the model's predict_proba attribute to find the set of loans which pass a threshold of 5%, 10%, 15%, ... 95%. For each such set, compute its volume (i.e. sum over loan_amnt) and its IRR. Create a curve which shows the volume vs IRR of each choice of threshold.

Now this graph shows us the tradeoff between the return and the deployed volume. Given this graph we are left with the buisness question of where along this tradeoff we wish to be, which will determine the threshold we should choose.

# Now share your model 

Now that your model is ready, it's time to create a package which streamlines it's usage.
Do the following:
* Inside the `pagayapro` repo, create a branch called `feature/{your_names}/co_classifier/`
* Inside `~/pagayapro/assignment3/`, create a folder called `models/{your_names}/` and in it a python package which contains a class for the model (you may choose the class's name). The class should have the attributes `fit`, `predict` and `predict_proba`.
* The attribute `fit` should accept two variable, X and y, preprocess them according to the steps you've undertaken in this notebook and fit a logistic regression model to the preprocessed data.
* The attribute `predict` should accept a test set X_test, preprocess it accrodingly, and return an array of boolean prediction for each row of X_test- True for CO and False for non-CO. The attribute `predict_proba` return the predicted probability of each row.
* Once you package is ready, push it to the repo. Ask your mentor for a review. Once the review is complete, rebase and merge your package to the repo.

# One last remark

An important thing we ignored in this exercise is that increased risk is not necessarily a bad thing, as a higher interest rate could compensate for it, with a premium. A portfolio that has 10% CO might be better than one that has 5% CO, if the average interest rate of the former is 20% and 10% for the latter. In the next assignment, we will attempt to use this observation to further improve our ability to choose a good portfolio.